In [ ]:
import pandas as pd

ex_tables = {
    "別表:基本的臨床手技": {"path":"./raw/sheets/技能編集用/基本的臨床手技.csv"},
    "別表:基本的診療科": {"path":"./raw/sheets/技能編集用/基本的診療科.csv"},
}

output=""

for table_name,info in ex_tables.items():
    table = pd.read_csv(info["path"])
    output += f"\n\n## {table_name}\n\n\n"+table.to_html()

table = pd.read_csv("./raw/sheets/知識編集用/臓器別知識.csv")

columns=list(table.columns.values)
temp_column=table.columns.values[-1]+"-temp"
table[temp_column]=table[table.columns.values[-1]]
table = table.groupby(columns).agg(lambda x:"".join(list(x)))
table=table.drop(temp_column,axis=1)
output += f"\n\n## 臓器別知識\n\n"+table.to_html()

output